### 수수료 엑셀을 읽어 지급율 row를 만든다

In [ ]:
import pandas as pd
import GladUtil as gd
COMP_LIFE = {'흥국생명':'흥국','한화생명':'한화','푸르덴셜':'푸르','카디프생명':'카디프','처브라이프':'처브','오렌지라이프':'오렌지','신한생명':'신한','삼성생명':'삼성',
             '미래에셋':'미래','메트라이프':'메트','라이나생명':'라이나','동양생명':'동양','농협생명':'농협','교보생명':'교보','KDB생명':'kdb','KB생명':'KB','IBK':'IBK',
             'DGB생명':'DGB','DB생명':'DB','ABL생명':'ABL'}
COMP_LONG = {'흥국화재':'흥국','한화손보':'한화','현대해상':'현대','삼성화재':'삼성','농협손보':'농협','DB손보':'DB','KB손보':'KB','메리츠손보':'메리츠',
             '롯데손보':'롯데','MG손보':'MG','AIG손보':'AIG','처브손보':'처브','하나손보':'하나','DAS':'DAS','chubb손보':'처브','메리츠화재':'메리츠'}
DICT_JONG = {'인보장':'보장','재물성':'재물','연금성':'연금','저축성':'저축','재물10년':'재물','연금10년':'연금','일반보장1':'보장','보장성':'보장'}
DICT_LIFE = {'변액연금': '변액연금1', '변액종신': '변액종신1', '변액적립': '변액적립1', '저축': '저축1', '연금': '연금1', '종신': '종신1',
              '보장성': '보장성1','실손':'실손1', '중저가(보장성4)': '중저가1'}

## 1. mp서식을 읽는 방법

In [ ]:
지점='표준'
대상='사원'
일괄적용율=1
보종='손보'
# 적용대상없으면 None으로 할 것
별도적용사= None # ['흥국생명','교보생명','KB생명']
별도적용율= 0.65  

path = 'D:/글로벌백업(180329)/03.수수료&시상/glad수수료규정/원수사지급율/'
# in_file = '경기_생손보수수료_202101.xlsx' #'2021년 생명보험 수수료_20210201.xlsx'
in_file='2021년 손해보험 수수료(20210210).xlsx'
sheet = '회사'

c = '회사 보종 환산율 상품명 1회 1차년율 1차년회차 2차년율 2차년회차 3차년율 3차년회차 tot'
columns = c.split(' ')
use_cols = 'a:d,e,g:h,l:m,q:r,v'
df = pd.read_excel(path+in_file,sheet_name=sheet,header=4,usecols=use_cols,dtype={'a': 'str', 'b': 'str'})
df.columns = columns
df['회사'].fillna(method='ffill',inplace=True)           
df['보종'] = df.보종.replace(0,None)
df['보종'].fillna(method='ffill',inplace=True)
df['tot'].fillna(method='ffill',inplace=True)
df.loc[:,'회사'] = df['회사'].str.replace('\n','')
df.fillna(0,inplace=True )

''' 서식에 대한 전처리'''
key_col = ['회사','보종','tot','1회']
df_t = df[key_col].copy()
df_t.loc[:,'회차']='1회'

key_col = ['회사','보종','tot','회차','율']
df_t.rename(columns={'1회':'율'},inplace=True)
df_rate = df_t[key_col]

key_col = ['회사','보종','tot','1차년회차','1차년율']
df_t = df[key_col].rename(columns={'1차년회차':'회차','1차년율':'율'})
df_rate = pd.concat([df_rate,df_t])

key_col = ['회사','보종','tot','2차년회차','2차년율']
df_t = df[key_col].rename(columns={'2차년회차':'회차','2차년율':'율'})
df_rate = pd.concat([df_rate,df_t])

key_col = ['회사','보종','tot','3차년회차','3차년율']
df_t = df[key_col].rename(columns={'3차년회차':'회차','3차년율':'율'})
df_rate = pd.concat([df_rate,df_t])

df_rate = df_rate[df_rate['율']!=0]
df_rate['회사율'] = df_rate['율']
df_rate['율'] = round(df_rate['율']*일괄적용율,4)
if  별도적용사:
    temp = df_rate.loc[df['회사'].isin(별도적용사),'회사율'] * 별도적용율
    df_rate.loc[df['회사'].isin(별도적용사),"율"] = temp
    
df_rate.loc[:,'회차'] = [str(x).replace('회','') for x in df_rate['회차']] 

'''각 회차별 생성'''
df_all = pd.DataFrame()
for i,row in df_rate.iterrows():
    for cnt in str(row['회차']).split(','):
        cnt1 = cnt;  cnt2 = cnt
        if '~' in cnt:
            cnt1,cnt2 = cnt.split('~') 
        row['회차'] = cnt1
        row['회차T'] = cnt2
        df_all = df_all.append(row)

In [ ]:
'''출력을 위한 파생항목 생성'''
df_rate_tbl = df_all
df_rate_tbl = df_rate_tbl.astype({'회차':'int64','회차T':'int64'})
df_rate_tbl['구분'] = 대상
df_rate_tbl['지점'] = 지점
df_rate_tbl = df_rate_tbl[['구분','회사','보종','율','회차','회차T','지점','회사율','tot']]

df_rate_tbl.sort_values(by=['회사','보종','회차','회차T'],ascending=[False,True,True,True],inplace=True)
df_rate_tbl['temp'] = df_rate_tbl['회사율'] * (df_rate_tbl['회차T'] - df_rate_tbl['회차'] + 1)
df_rate_tbl['tot_compare'] = df_rate_tbl.groupby(['회사','보종']).temp.transform('sum')
df_rate_tbl.drop('temp',axis=1, inplace=True)

# 생보의 경우 보종명 맨뒤에 숫자가 없으면 1을 붙임
# if 보종=="생보":
#     num_list = [ str(i) for i in range(1,10)]
#     df_rate_tbl['보종'] = [ i if i[len(i)-1] in num_list else i +'1' for i in df_rate_tbl['보종']]

'''tot와 tot_compare가 다르면 엑셀 수식오류 검증해봐야함'''

outFile = f'd:/temp/수수료표/{지점}_{보종} 지급율-01.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_rate_tbl.to_excel(writer, sheet_name='지급율',index=False)

### 1.1) ibss업로드용 엑셀 생성(요율구간은 7개가 최대임)

In [ ]:
## 회사명을 IBSS에 맞게 변경
if 보종 == '생보':
    df_rate_tbl['회사'] = df_rate_tbl['회사'].apply(lambda x: gd.convertByVal(x,COMP_LIFE))
else:
    df_rate_tbl['회사'] = df_rate_tbl['회사'].apply(lambda x: gd.convertByVal(x,COMP_LONG))
    df_rate_tbl['보종'] = df_rate_tbl['보종'].apply(lambda x: gd.convertByVal(x,DICT_JONG))

dfr = df_rate_tbl.loc[:,['회사','보종','회차','회차T','율']].copy()
dfr.loc[:,'율'] = dfr.율 * 100     #업로드용 서식으로 환산
dfr = dfr.set_index(['회사','보종'])

# 컬럼을 7쌍 생성
cols = [  [f'기간{i}-시작',f'기간{i}-끝',f'기간{i}-요율'] for i in range(1,11)]
bf_i = ()
dft = pd.DataFrame()
df_rate_upload = pd.DataFrame()
col_idx = 0

for idx, row in dfr.iterrows():
    if bf_i == idx:   #df index가 같으면 컬럼을 옆으로 붙임
        col_idx += 1
        if col_idx > 6:
            print('요율범위초과 ', idx , col_idx+1 , '번째')

        col_name = cols[col_idx]
        t_from = row.values[0]
        t_to = row.values[1]
        t_rate = row.values[2]
        dft[col_name[0]] = t_from
        dft[col_name[1]] = t_to
        dft[col_name[2]] = t_rate
    else:
        df_rate_upload = pd.concat([df_rate_upload,dft])
        col_idx = 0
        dft = pd.DataFrame(data = [row.values], columns = cols[col_idx], index = [[idx[0]],[idx[1]]] )
    bf_i = idx
    
df_rate_upload = pd.concat([df_rate_upload,dft])
df_rate_upload = df_rate_upload.reset_index()
df_rate_upload = df_rate_upload.rename(columns={'level_0':'보험사','level_1':'정산타입'})

outFile = f'd:/temp/수수료표/{지점}_{보종} 지급율_upload-{일괄적용율}.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_rate_upload.to_excel(writer, sheet_name='Worksheet',index=False)
    
## 보험회사명과 정산타입(보종군), 템플릿 헤더 작업이 추가로 필요함

### 1.2) 아래는 mp서식읽은 결과로 glad방식(회차별로 column)을 만드는 작업

In [ ]:
df_all = pd.DataFrame()
for i,row in df_rate.iterrows():
    for cnt in str(row['회차']).split(','):
        cnt1 = cnt;  cnt2 = cnt
        if '~' in cnt:
            cnt1,cnt2 = cnt.split('~') 
        for x in range(int(cnt1),int(cnt2)+1):
            row['회차'] = x
            df_all=df_all.append(row)

df_rate_tbl = df_all
df_rate_tbl = df_rate_tbl.astype({'회차':'int64'})

df_rate_tbl['구분'] = 대상
df_rate_tbl['지점'] = 지점
df_rate_tbl = df_rate_tbl[['구분','회사','보종','율','회차']]

df_rate_tbl.sort_values(by=['회사','보종'],ascending=[False,True],inplace=True)
df_rate_tbl = pd.pivot_table(df_rate_tbl,index=['구분','회사','보종'],columns='회차',values=['율'])
df_rate_tbl = df_rate_tbl.fillna(0)            
df_rate_tbl.reset_index(inplace=True)
df_rate_tbl.columns = df_rate_tbl.columns.get_level_values(1) #multi index -> single index

outFile=f'd:/temp/수수료표/{지점}_{보종} 회차별지급율.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_rate_tbl.to_excel(writer, sheet_name='지급율',index=False)

### 수수료 파일 비교

In [ ]:
''' 두개의 수수료 파일 총 지급율을 비교하여 차이나는 건 추출'''
dt2=df_rate2.groupby(['회사','보종'])['회사율'].sum().reset_index()
dt1=df_rate1.groupby(['회사','보종'])['회사율'].sum().reset_index()
dt_merge= pd.merge(dt2,dt1,on=['회사','보종'],how='left')
dt_merge[dt_merge['회사율_x'] != dt_merge['회사율_y']]

## 2.글로벌/glad표준파일 처리 → 수수료검증엑셀용(회사/보종/회차from~to)파일

In [54]:
## glad표준 엑셀 파일
지점 = '표준'
대상 = '사원'
일괄적용율 = 0.65
보종 = '손보'

path ='d:/글로벌백업(180329)/03.수수료&시상/glad수수료규정/원수사지급율/'
in_file = '수수료(2021.4)선지급형_손보.xlsx'
sheet = '손해보험(회사)'
c = '회사 보종 상품명 납기 환산율 환산율2 환산율3 1회 수입P1 2회 3회 4회 5회 6회 7회 8회 9회 10회 11회 12회 지급계1y 수입p1y 13회 14회 15회 16회 17회 18회 19회 20회 21회 22회 23회 24회 지급계2y 수입P2y 25회 26회 27회 28회 29회 30회 31회 32회 33회 34회 35회 36회 지급계3y 수입P3y 지급계 수입P 비고'
columns = c.split(' ')
df = pd.read_excel(path+in_file,sheet_name=sheet,header=4,names=columns)
df['회사'].fillna(method='ffill',inplace=True)
df['보종'].fillna(method='ffill',inplace=True)
df.loc[:,'보종'] = df['보종'].str.replace('\n','').str.replace(' ','')
df.loc[:,'회사'] = df['회사'].str.replace('\n','').str.replace(' ','')
df.fillna(0,inplace=True )

In [55]:
# df=df.iloc[35:47]
# 컬럼중 회차별 지급율이 있는 컬럼만 추출-- 키워드는 '회'
value_col = [x for x in columns if x.endswith('회') ]

# 열 -> 행으로 바꿈
df_rate = df.melt(id_vars=['회사','보종','지급계'],value_vars=value_col,var_name='회차',value_name='율')
df_rate = df_rate[df_rate['율']!=0]     #지급율 없으면 제거
df_rate['회사율'] = df_rate['율']
df_rate['율'] = round(df_rate['율']*일괄적용율,4) #소수 4자리 반올림 
df_rate.loc[:,'회차'] = [str(x).replace('회','') for x in df_rate['회차']]
df_rate = df_rate.astype({'회차':'int64'})
df_rate['회차T'] = df_rate['회차'] 
df_rate['구분'] = 대상
df_rate['지점'] = 지점
df_rate = df_rate[['구분','회사','보종','율','회차','회차T','지점','회사율','지급계']]
df_rate.sort_values(by=['회사','보종','회차'],ascending=[False,True,True],inplace=True)

## 회차별지급율sum과 엑셀의 합계 지급율을 비교
df_rate.sort_values(by=['회사','보종','회차','회차T'],ascending=[False,True,True,True],inplace=True)
df_rate['temp'] = df_rate['회사율'] * (df_rate['회차T'] - df_rate['회차'] + 1)
df_rate['tot_compare'] = df_rate.groupby(['회사','보종']).temp.transform('sum')
df_rate.drop('temp',axis=1, inplace=True)
df_rate.rename(columns={'지급계':'tot'},inplace=True)

outFile = f'd:/temp/수수료표/{지점}{보종} 지급율_{일괄적용율}.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_rate.to_excel(writer, sheet_name='지급율',index=False)

## 3.글로벌/glad표준파일 → ibss업로드 양식으로 만든다(회사/보종/회차from~to(7개))

In [ ]:
## glad표준 엑셀 파일
지점 = '표준'
대상 = '사원'
일괄적용율 = 0.94
보종 = '생보'
# 적용대상없으면 None으로 할 것
별도적용사= None #['흥국','교보','KB']#['흥국생명','교보생명','KB생명']  #None
별도적용율= 0.65

# path ='d:/글로벌백업(180329)/03.수수료&시상/수수료규정/수수료개정_2021/지점별/'
# in_file = '엠피조은_생손보수수료_202101.xlsx'
# sheet = '엠피조은- 손보 - 이설화,김성희 '

path ='C:/Users/User/Desktop/myga/'
in_file = '수수료(2021.4)선지급형_생보.xlsx'
sheet = '생명보험(회사)'

c = '회사 보종 상품명 납기 환산율 환산율2 환산율3 1회 수입P1 2회 3회 4회 5회 6회 7회 8회 9회 10회 11회 12회 지급계1y 수입p1y 13회 14회 15회 16회 17회 18회 19회 20회 21회 22회 23회 24회 지급계2y 수입P2y 25회 26회 27회 28회 29회 30회 31회 32회 33회 34회 35회 36회 지급계3y 수입P3y 지급계 수입P 비고'
columns = c.split(' ')
df = pd.read_excel(path+in_file,sheet_name=sheet,header=4,names=columns)
df['회사'].fillna(method='ffill',inplace=True)
df['보종'] = df.보종.replace(0,None)
df['보종'].fillna(method='ffill',inplace=True)
df.loc[:,'보종'] = df['보종'].str.replace('\n','').str.replace(' ','')
df.loc[:,'회사'] = df['회사'].str.replace('\n','').str.replace(' ','')
df.fillna(0,inplace=True )

## 회사명,보종명을 IBSS에 맞게 변경
df['납기'] = df['납기'].astype(str)
df['보납'] = df['보종']+'('+df['납기']+'년)'

if 보종 == '생보':
    df['회사'] = df['회사'].apply(lambda x: gd.convertByVal(x,COMP_LIFE))
    temp = df.loc[df['회사'].isin(['IBK'])] ['보납']
    df.loc[df['회사'].isin(['IBK']),'보종']= temp
    df['보종'] = df['보종'].apply(lambda x: gd.convertByVal(x,DICT_LIFE))
else:
    df['회사'] = df['회사'].apply(lambda x: gd.convertByVal(x,COMP_LONG))
    df['보종'] = df['보종'].apply(lambda x: gd.convertByVal(x,DICT_JONG))

## ibss용 컬럼을 생성
cols = [  [f'기간{i}-시작',f'기간{i}-끝',f'기간{i}-요율'] for i in range(1,11)]
cols = ['회사','보종'] + sum(cols,[])
if 보종 == '손보':
    cols.insert(2,'실손시작회차')

In [ ]:
# 각 컬렁을 순회하면서 지급율이 같으면 from-to를 만듬
def make_rate_fromto(row):
    lis = []
    bf_val = 0
    tl = []
    for idx,val in row.items():
        if idx.endswith('회'):
#             val = round(float(val) * 100,2)                # %를 100단위로 환산-소수점2자리에서 반올림
            val = float(val)
            cnt = int(idx.replace("회",''))
            if cnt == 1:         #1회차는 출력
                if val !=0:
                    lis.append([1, 1, val])
                continue

            if  val != 0:
                if val == bf_val:
                    tl[1] = cnt
                    tl[2] = val
                else:
                    if  tl: 
                        lis.append(tl)
                        tl = [] 

                    tl.append(cnt)
                    tl.append(cnt)
                    tl.append(val)
            bf_val = val
   
    if tl: 
        lis.append(tl)

    tot = sum([ (i[1] - i[0] + 1) * i[2] for i in lis ])
    lis = sum(lis,[])         #2차원리스트를 1차원으로 변경
    cnt = 30 - len(lis)
    lis.extend([None]*cnt)    #list개수를 30개로 확장
    return lis, tot

# make_rate_fromto(df.iloc[52])

In [ ]:
dft = pd.DataFrame()
df_rate_upload = pd.DataFrame(columns = cols)

for i,row in df.iterrows():
    tot1 = float(row['지급계']) * 1
    lis, tot = make_rate_fromto(row)
    if abs(tot - tot1 ) > 0.1:
        print( f"오류건 =  {row['회사']} {row['보종']}  회차지급율 합({tot})  <> 엑셀 지급계({tot1})")

    lis.insert(0,row['회사'])
    lis.insert(1,row['보종'])
    if 보종 == '손보':
        lis.insert(2,'')
    
    df_rate_upload.loc[len(df_rate_upload)] = lis

# 요율컬럼에만 계산 실시 -> 최종적으로 %를 100단위로 환산
cols_r = list(filter(lambda x:  x.endswith('요율'), cols))
df_rate_upload[cols_r] = df_rate_upload[cols_r] * 일괄적용율
if  별도적용사:
    temp = df_rate_upload.loc[df['회사'].isin(별도적용사),cols_r] /일괄적용율 * 별도적용율
    df_rate_upload.loc[df['회사'].isin(별도적용사),cols_r] = temp
    
df_rate_upload[cols_r] = df_rate_upload[cols_r].astype(float)  # 명시적으로 float type으로 바꿔야 round함수가 잘 먹힘
df_rate_upload[cols_r] = round(df_rate_upload[cols_r] * 100, 2)
    
outFile = f'd:/temp/수수료표/IBSS-upload-{보종}-{지점}{대상} 지급율_{일괄적용율}.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_rate_upload.to_excel(writer, sheet_name='Worksheet',index=False)

### 아래는 경기 (신은경) 지급율 생성용

In [ ]:

dft = pd.DataFrame()
df_rate_upload = pd.DataFrame(columns = cols)

for i,row in df.iterrows():
    tot1 = float(row['지급계']) * 1
    lis, tot = make_rate_fromto(row)
    if abs(tot - tot1 ) > 0.1:
        print( f"오류건 =  {row['회사']} {row['보종']}  회차지급율 합({tot})  <> 엑셀 지급계({tot1})")

    lis.insert(0,row['회사'])
    lis.insert(1,row['보종'])
    if 보종 == '손보':
        lis.insert(2,'')
    
    df_rate_upload.loc[len(df_rate_upload)] = lis
    
cols_r = list(filter(lambda x:  x.endswith('요율'), cols))
cols_r.pop(0)
df_rate_upload[cols_r] = df_rate_upload[cols_r] * 0.65
df_rate_upload['기간1-요율'] = df_rate_upload['기간1-요율'] * 0.8
cols_r = list(filter(lambda x:  x.endswith('요율'), cols))
df_rate_upload[cols_r] = df_rate_upload[cols_r].astype(float)  # 명시적으로 float type으로 바꿔야 round함수가 잘 먹힘
df_rate_upload[cols_r] = round(df_rate_upload[cols_r] * 100, 2)
outFile = f'd:/temp/수수료표/IBSS-upload-{보종}-{지점}{대상} 지급율_{일괄적용율}.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_rate_upload.to_excel(writer, sheet_name='Worksheet',index=False)

### 임시 사용 스크립트

In [ ]:
## glad표준 엑셀 파일(아이리치)--임시용
지점='아이리치'
대상='사원'
일괄적용율=1.0
보종='손보'

path='E:/temp/'
in_file='book1.xlsx'
sheet='Sheet1'
c='회사 보종 환산율 1회 7회 8회 9회 10회 11회 12회 13회 14회 15회 지급계'
columns=c.split(' ')
df=pd.read_excel(path+in_file,sheet_name=sheet,header=0,names=columns)
df['회사'].fillna(method='ffill',inplace=True)
df.fillna(0,inplace=True )

## 3. 보종명 변환

In [ ]:
path='E:/temp/'
in_file='2018기준율-생보-변환전.xlsx'
sheet='Sheet1'

c='회사 보종 환산율 상품명 1회 1차년율 1차년회차 2차년율 2차년회차 3차년율 3차년회차 tot'
df=pd.read_excel(path+in_file,sheet_name=sheet)
df=df.fillna('')

In [ ]:
b_list=['일반보장','일반종신','변액종신','연금','변액연금','저축','일반연금','변액적립','중저가',
        '보장','종신','보장2','보장3','보장4','경영인정기','적립','변액','양한방',
        '일반보장-경영','연금-7년','연금-5년','일반보장(경영인)','']
c_list=['보장성1','종신1','변액종신1','연금1','변액연금1','저축1','연금1','변액적립1','중저가1',
        '보장성1','종신1','보장성2','보장성3','보장성4','보장성1','변액적립1','변액연금1','양한방',
        '일반보장-경영','연금-7년','연금-5년','일반보장(경영인)','']
conv_dict=dict(zip(b_list,c_list))

In [ ]:
lis=[5,15,25,35,45,55,65,75,85]
for i in lis:
#     print(i)
    df[i]=df[i].map(lambda x: conv_dict[x])